<a href="https://colab.research.google.com/github/verma-saloni/Articles/blob/master/5_5_22_3_Fasttext_Embeddings_XGboost_Politifact_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#%%capture
!pip install -U xgboost wandb fasttext pyfasttext

     |████████████████████████████████| 193.7 MB 32 kB/s 
     |████████████████████████████████| 1.8 MB 33.4 MB/s 
     |████████████████████████████████| 68 kB 5.4 MB/s 
     |████████████████████████████████| 244 kB 54.9 MB/s 
     |████████████████████████████████| 181 kB 46.9 MB/s 
     |████████████████████████████████| 144 kB 43.3 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Using cached cysignals-1.11.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (871 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3147506 sha256=8e55ac732fad33f81d36a513871f339fe14558dd7d20217b621959c0a9d17b8b
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
  Created wheel for pyfasttext: filename=pyfasttext-0.4.6-cp37-cp37m-linux_x86_64.whl size=1460707 sha256=7f4c7095e760b0757bf26cc00f339bfed04c7e07b21e2bd39f4c64be7de

In [2]:
import pandas as pd
import numpy as np
import nltk
import gensim
from gensim.models import Word2Vec, FastText
import fasttext as ft
import string
nltk.download('stopwords')
import re
import spacy
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import xgboost as xgb
from pathlib import Path

import wandb
from wandb.xgboost import WandbCallback
from IPython.display import clear_output


metrics = [accuracy_score, f1_score, precision_score, recall_score]

def get_name(score_func):
    return score_func.__name__.split("_")[0]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
base_dir = Path("/content/drive/MyDrive/Research FakeNews")
#assert base_dir.exists()

In [5]:
df = pd.read_csv(base_dir/"politifact_agg.csv", index_col=0)

In [6]:
df.head()

,title,text,tweets,retweets,label,url,tweet_ids,num_retweets,log_num_retweets,num_tweets,log_num_tweets
0,Actress Emma Stone ‘For the first time in his...,NaN,[],"['1020554564334964741', '1020817527046197248',...",fake,NaN,[],2911,7.976595,0,0.0
1,Breaking President Trump makes English the of...,NaN,[],[],fake,NaN,[],0,0.000000,0,0.0
2,Friendly Fire … Charlottesville Car attacker ...,NaN,[],"['3265439004', '3250621593', '3253922920', '32...",fake,NaN,[],24,3.218876,0,0.0
3,If You Are Using This Toothpaste… Throw It Aw...,MightyLiving Blog\n\nHelpful inspiration from ...,[],"['911971426571255810', '1036749614853103616', ...",fake,mightynest.com/learn/research-by-concern/dange...,[],2569,7.851661,0,0.0
4,"""Face the Nation"" transcripts, August 26, 2012...","""Face the Nation"" transcripts, August 26, 2012...",[],[],real,https://web.archive.org/web/20120827001956/htt...,[],0,0.000000,0,0.0


In [7]:
# NLP Preprocessing
from gensim.utils import simple_preprocess

# NLP Preprocess - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15)[source]
# Convert a document into a list of tokens.
# This lowercases, tokenizes, de-accents (optional). – the output are final tokens = unicode strings, that won’t be processed any further.

df["title"] = df.title.map(lambda x: ' '.join(simple_preprocess(x)))

In [8]:
#%%capture
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
!gunzip cc.en.300.bin.gz

--2022-05-04 22:39:09--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G  35.0MB/s    in 99s     

2022-05-04 22:40:48 (43.4 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]



In [34]:
# Loading model for English
modelog = ft.load_model('cc.en.300.bin') 

In [29]:
ft_title_emb_file = base_dir/"ft_title_embeddings.npy"

if ft_title_emb_file.exists():
    title_embeddings = np.load(ft_title_emb_file)
else:
    title_embeddings = []
    for line in df['title']:
        emb = modelog.get_sentence_vector(line)
        title_embeddings.append(emb)
    title_embeddings = np.stack(title_embeddings)
    np.save(ft_title_emb_file, title_embeddings)

In [30]:
ft_text_emb_file = base_dir/"ft_text_embeddings.npy"

if ft_text_emb_file.exists():
    text_embeddings = np.load(ft_text_emb_file)
else:
    df.text.fillna("", inplace=True)
    texts = df.title + " " + df.text.map(lambda x: " ".join(simple_preprocess(x)))
    text_embeddings = []
    for line in texts:
        emb = modelog.get_sentence_vector(line)
        text_embeddings.append(emb)
    text_embeddings = np.stack(text_embeddings)
    np.save(ft_text_emb_file, text_embeddings)

Will be attempting xgboost here.

In [35]:
import gc

del modelog; gc.collect()

262

### Title embeddings

In [36]:
X = title_embeddings
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((894, 300), (894,))

In [37]:
skf = StratifiedKFold(shuffle=True, random_state=124)

In [38]:
def train(train_idx, test_idx, params):

    # training
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    watchlist = [(dtrain,'train'), (dtest,'eval')]
    clf = xgb.train(params, dtrain, num_boost_round=100, early_stopping_rounds=None, evals=watchlist, callbacks=[WandbCallback()])
    #evaluation
    probs = clf.predict(dtest)
    y_pred = (probs > 0.5).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})

In [39]:
params = {
    "objective":'binary:logistic',
    "seed":124
}

In [40]:
GROUP = "fasttext-title-xgb"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity="saloniteam", project="fnd", group=GROUP, name=f"{GROUP}-fold-{fold_id}") as run:
        train(train_idx, test_idx, params)

[0]	train-logloss:0.51851	eval-logloss:0.61572
[1]	train-logloss:0.40812	eval-logloss:0.58951
[2]	train-logloss:0.32666	eval-logloss:0.55907
[3]	train-logloss:0.26414	eval-logloss:0.53379
[4]	train-logloss:0.21567	eval-logloss:0.50541
[5]	train-logloss:0.18021	eval-logloss:0.49854
[6]	train-logloss:0.15367	eval-logloss:0.49576
[7]	train-logloss:0.13064	eval-logloss:0.48193
[8]	train-logloss:0.11177	eval-logloss:0.47289
[9]	train-logloss:0.09766	eval-logloss:0.47519
[10]	train-logloss:0.08434	eval-logloss:0.46956
[11]	train-logloss:0.07409	eval-logloss:0.46545
[12]	train-logloss:0.06489	eval-logloss:0.46792
[13]	train-logloss:0.05881	eval-logloss:0.46983
[14]	train-logloss:0.05286	eval-logloss:0.47294
[15]	train-logloss:0.04839	eval-logloss:0.47244
[16]	train-logloss:0.04411	eval-logloss:0.47571
[17]	train-logloss:0.04038	eval-logloss:0.47395
[18]	train-logloss:0.03704	eval-logloss:0.47463
[19]	train-logloss:0.03435	eval-logloss:0.48215
[20]	train-logloss:0.03209	eval-logloss:0.48404
[2

accuracy,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▅▂▂▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▅
f1,▁
precision,▁
recall,▁
train-logloss,█▅▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.76966
epoch,99
f1,0.78534
precision,0.78947


### Text embeddings

In [41]:
X = text_embeddings
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((894, 300), (894,))

In [42]:
skf = StratifiedKFold(shuffle=True, random_state=124)

In [43]:
def train(train_idx, test_idx, params):

    # training
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    watchlist = [(dtrain,'train'), (dtest,'eval')]
    clf = xgb.train(params, dtrain, num_boost_round=100, early_stopping_rounds=None, evals=watchlist, callbacks=[WandbCallback()])
    #evaluation
    probs = clf.predict(dtest)
    y_pred = (probs > 0.5).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})

In [ ]:
params = {
    "objective":'binary:logistic',
    "seed":124
}

In [ ]:
GROUP = "fastext-fulltext-xgb"
for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity="saloniteam", project="fnd", group=GROUP, name=f"{GROUP}-fold-{fold_id}") as run:
        train(train_idx, test_idx, params)

### + num tweets and retweets

In [ ]:
X = np.concatenate([text_embeddings, df.num_retweets.to_numpy()[..., None], df.num_tweets.to_numpy()[..., None]], axis=1)
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

In [ ]:
skf = StratifiedKFold(shuffle=True, random_state=124)

In [ ]:
def train(train_idx, test_idx, params):

    # training
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    watchlist = [(dtrain,'train'), (dtest,'eval')]
    clf = xgb.train(params, dtrain, num_boost_round=100, early_stopping_rounds=None, evals=watchlist, callbacks=[WandbCallback()])
    #evaluation
    probs = clf.predict(dtest)
    y_pred = (probs > 0.5).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})

In [ ]:
params = {
    "objective":'binary:logistic',
    "seed":124
}

In [ ]:
GROUP = "fastext-fulltext-tw-xgb"
for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity="saloniteam", project="fnd", group=GROUP, name=f"{GROUP}-fold-{fold_id}") as run:
        train(train_idx, test_idx, params)

Next step, keras NN model.